<a href="https://colab.research.google.com/github/rushithakondreddy/haihua_INFO5731_Spring2020/blob/main/INFO5731_Assignment_Two.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **INFO5731 Assignment Two**

In this assignment, you will try to gather text data from open data source via web scraping or API. After that you need to clean the text data and syntactic analysis of the data.

# **Question 1**

(40 points). Write a python program to collect text data from **either of the following sources** and save the data into a **csv file**:

(1) Collect all the customer reviews of the product [2019 Dell labtop](https://www.amazon.com/Dell-Inspiron-5000-5570-Laptop/dp/B07N49F51N/ref=sr_1_11?crid=1IJ7UWF2F4GHH&keywords=dell%2Bxps%2B15&qid=1580173569&sprefix=dell%2Caps%2C181&sr=8-11&th=1) on amazon.

(2) Collect the top 100 User Reviews of the film [Joker](https://www.imdb.com/title/tt7286456/reviews?ref_=tt_urv) from IMDB.

(3) Collect the abstracts of the top 100 research papers by using the query [natural language processing](https://citeseerx.ist.psu.edu/search?q=natural+language+processing&submit.x=0&submit.y=0&sort=rlv&t=doc) from CiteSeerX.

(4) Collect the top 100 tweets by using hashtag ["#CovidVaccine"](https://twitter.com/hashtag/CovidVaccine) from Twitter. 


In [1]:
# Write your code here

import requests
from urllib.parse import urljoin
from bs4 import BeautifulSoup as BS
import re
import numpy as np
import pandas as pd

def get_tablelist(soup):
    reviews = soup.find_all("div", {"class": "lister-item-content"})

    review_data = []

    for review in reviews:

        try:
            review_content = review.find("div", {"class": "text show-more__control"}).text.strip()
        except:
            review_content = review.find("div", {"class": "text show-more__control clickable"}).text.strip()

        review_data.append(review_content)
    return review_data

b_url = 'https://www.imdb.com/'
url = 'https://www.imdb.com/title/tt7286456/reviews?ref_=tt_urv'
res = requests.get(url)
soup = BS(res.text, 'html.parser')
res.encoding = 'utf-8'
allTablelist=[]
tablelist = get_tablelist(soup)
for i in tablelist:
  allTablelist.append(i)

load_more = soup.select(".load-more-data")
flag = True
if len(load_more):
    ajaxurl = load_more[0]['data-ajaxurl']
    b_url = b_url + ajaxurl + "?ref_=undefined&paginationKey="
    try:
        key = load_more[0]['data-key']
    except KeyError:
        flag = False
else:
    flag = False

while flag:
    url = b_url + key
    res = requests.get(url)
    res.encoding = 'utf-8'
    soup = BS(res.text, 'html.parser')
    tablelist2 = get_tablelist(soup)
    if len(allTablelist) == 100:
      break
    for i in tablelist2:
        allTablelist.append(i)
    load_more = soup.select(".load-more-data")
    if len(load_more):
        key = load_more[0]['data-key']
    else:
        flag = False

data = np.array(allTablelist)
data = pd.DataFrame(data, columns=['review_content'])
data.shape
data.to_csv('review_data.csv')
data





,review_content
0,"Every once in a while a movie comes, that trul..."
1,This is a movie that only those who have felt ...
2,"Truly a masterpiece, The Best Hollywood film o..."
3,Joaquin Phoenix gives a tour de force performa...
4,Most of the time movies are anticipated like t...
...,...
95,This film tries desperately to devillify the m...
96,"Wow I honestly gotta tell you, it's one of the..."
97,The whole point of this movie is to bring atte...
98,"While I enjoyed the film, it felt pretty short..."


# **Question 2**

(30 points). Write a python program to **clean the text data** you collected above and save the data in a new column in the csv file. The data cleaning steps include:

(1) Remove noise, such as special characters and punctuations.

(2) Remove numbers.

(3) Remove stopwords by using the [stopwords list](https://gist.github.com/sebleier/554280).

(4) Lowercase all texts

(5) Stemming. 

(6) Lemmatization.

In [2]:
#converting to lowercase

data['review_content'] = data['review_content'].apply(lambda x: " ".join(x.lower() for x in x.split()))
data['review_content'].head()





0    every once in a while a movie comes, that trul...
1    this is a movie that only those who have felt ...
2    truly a masterpiece, the best hollywood film o...
3    joaquin phoenix gives a tour de force performa...
4    most of the time movies are anticipated like t...
Name: review_content, dtype: object

In [4]:
#remove punct
data['review_content'] = data['review_content'].str.replace('[^\w\s]','')
data['review_content'].head()

0    every once in a while a movie comes that truly...
1    this is a movie that only those who have felt ...
2    truly a masterpiece the best hollywood film of...
3    joaquin phoenix gives a tour de force performa...
4    most of the time movies are anticipated like t...
Name: review_content, dtype: object

In [5]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [6]:
# remove stopwords
from nltk.corpus import stopwords
stop = stopwords.words('english')
data['review_content'] = data['review_content'].apply(lambda x: " ".join(x for x in x.split() if x not in stop))
data['review_content'].head()

0    every movie comes truly makes impact joaquins ...
1    movie felt alone isolated truly relate underst...
2    truly masterpiece best hollywood film 2019 one...
3    joaquin phoenix gives tour de force performanc...
4    time movies anticipated like end falling short...
Name: review_content, dtype: object

In [7]:
#remove special characters
import string
string.punctuation


'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [8]:

#remove special characters
data['review_content'] = data['review_content'].apply(lambda x: " ".join(x.strip(string.punctuation) for x in x.split()))  
data['review_content']

0     every movie comes truly makes impact joaquins ...
1     movie felt alone isolated truly relate underst...
2     truly masterpiece best hollywood film 2019 one...
3     joaquin phoenix gives tour de force performanc...
4     time movies anticipated like end falling short...
                            ...                        
95    film tries desperately devillify stylish chari...
96    wow honestly gotta tell one best movies ive se...
97    whole point movie bring attention mental illne...
98    enjoyed film felt pretty short end appeared sc...
99    homage borrowing wholesale material film littl...
Name: review_content, Length: 100, dtype: object

In [9]:
#remove numbers
data['review_content'] = data['review_content'].apply(lambda x: " ".join(x for x in x.split() if not x.isdigit()))
data['review_content']

0     every movie comes truly makes impact joaquins ...
1     movie felt alone isolated truly relate underst...
2     truly masterpiece best hollywood film one best...
3     joaquin phoenix gives tour de force performanc...
4     time movies anticipated like end falling short...
                            ...                        
95    film tries desperately devillify stylish chari...
96    wow honestly gotta tell one best movies ive se...
97    whole point movie bring attention mental illne...
98    enjoyed film felt pretty short end appeared sc...
99    homage borrowing wholesale material film littl...
Name: review_content, Length: 100, dtype: object

In [10]:
#spell correction
from textblob import TextBlob
data['review_content'][:5].apply(lambda x: str(TextBlob(x).correct()))

0    every movie comes truly makes impact joaquins ...
1    movie felt alone isolated truly relate underst...
2    truly masterpiece best hollywood film one best...
3    joaquin phoenix gives tour de force performanc...
4    time moves anticipated like end falling short ...
Name: review_content, dtype: object

In [11]:
#stemming
from nltk.stem import PorterStemmer
st = PorterStemmer()
data['review_content'][:5].apply(lambda x: " ".join([st.stem(word) for word in x.split()]))

0    everi movi come truli make impact joaquin perf...
1    movi felt alon isol truli relat understand mot...
2    truli masterpiec best hollywood film one best ...
3    joaquin phoenix give tour de forc perform fear...
4    time movi anticip like end fall short way shor...
Name: review_content, dtype: object

In [12]:
#lemmatization
from textblob import Word
nltk.download('wordnet')
data['review_content'] = data['review_content'].apply(lambda x: " ".join([Word(word).lemmatize() for word in x.split()]))
data['review_content'].head()

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


0    every movie come truly make impact joaquins pe...
1    movie felt alone isolated truly relate underst...
2    truly masterpiece best hollywood film one best...
3    joaquin phoenix give tour de force performance...
4    time movie anticipated like end falling short ...
Name: review_content, dtype: object

# **Question 3**

(30 points). Write a python program to conduct **syntax and structure analysis** of the clean text you just saved above. The syntax and structure analysis includes: 

(1) Parts of Speech (POS) Tagging: Tag Parts of Speech of each word in the text, and calculate the total number of N(oun), V(erb), Adj(ective), Adv(erb), respectively.

(2) Constituency Parsing and Dependency Parsing: print out the constituency parsing trees and dependency parsing trees of all the sentences. Using one sentence as an example to explain your understanding about the constituency parsing tree and dependency parsing tree.

(3) Named Entity Recognition: Extract all the entities such as person names, organizations, locations, product names, and date from the clean texts, calculate the count of each entity.

In [13]:
# Write your code here
nltk.download('averaged_perceptron_tagger')





[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [14]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [15]:
from nltk import word_tokenize, pos_tag, pos_tag_sents
data['POSTags'] =  pos_tag_sents( data['review_content'].apply(word_tokenize).tolist() )
print(data)

pos = data['POSTags'].to_list()
pos
nouns = 0
for x in pos:
  for a,b in x:
    if(b.startswith('N')):
       nouns+=1
res = "nouns:{}".format(nouns)
print(res)
verb = 0
for x in pos:
  for a,b in x:
    if(b.startswith('V')):
       verb+=1
res1 = "verb:{}".format(verb)
print(res1)
adverb = 0
for x in pos:
  for a,b in x:
    if(b.startswith('R')):
       adverb+=1
res2 = "adverb:{}".format(adverb)
print(res2)
adj = 0
for x in pos:
  for a,b in x:
    if(b.startswith('J')):
       adj+=1
res3 = "adj:{}".format(adj)
print(res3)

                                       review_content                                            POSTags
0   every movie come truly make impact joaquins pe...  [(every, DT), (movie, NN), (come, VBN), (truly...
1   movie felt alone isolated truly relate underst...  [(movie, NN), (felt, VBD), (alone, RB), (isola...
2   truly masterpiece best hollywood film one best...  [(truly, RB), (masterpiece, JJ), (best, JJS), ...
3   joaquin phoenix give tour de force performance...  [(joaquin, NN), (phoenix, NN), (give, VB), (to...
4   time movie anticipated like end falling short ...  [(time, NN), (movie, NN), (anticipated, VBN), ...
..                                                ...                                                ...
95  film try desperately devillify stylish charism...  [(film, NN), (try, VB), (desperately, RB), (de...
96  wow honestly gotta tell one best movie ive see...  [(wow, NN), (honestly, RB), (got, VBD), (ta, R...
97  whole point movie bring attention mental illne...  

In [16]:
#entity recognition
!pip install spacy

In [17]:
import spacy
from spacy import displacy
from collections import Counter
import en_core_web_sm
nlp = spacy.load("en_core_web_sm")
data['review_content']
list = data['review_content'].tolist()
list[0]
for x in list:
   doc = (nlp(x))
   print([(X.text, X.label_) for X in doc.ents])


[]
[('one', 'CARDINAL')]
[('hollywood', 'GPE'), ('one', 'CARDINAL')]
[('joaquin phoenix', 'PERSON'), ('de force', 'ORG'), ('portrayal', 'NORP')]
[('dark joker', 'ORG'), ('villain', 'GPE')]
[('joaquin phoneix', 'PERSON')]
[('arthur madness', 'PERSON'), ('phillips', 'PERSON')]
[('yesterday', 'DATE'), ('venice', 'GPE'), ('itjoker', 'PERSON'), ('venice', 'GPE'), ('nolans', 'NORP'), ('scorsese', 'NORP'), ('hollywood', 'GPE')]
[('joaquin', 'PERSON'), ('niro joaquin', 'PERSON')]
[('joker venice', 'PERSON'), ('joaquin', 'PERSON'), ('robert', 'PERSON')]
[('phoenix superb', 'ORG'), ('arthur mental state deep psychological', 'ORG'), ('one', 'CARDINAL'), ('three', 'CARDINAL')]
[]
[]
[]
[('joaquin phoenix', 'PERSON'), ('todd phillips', 'PERSON'), ('disturbed', 'ORG')]
[('joaquin phoenix', 'PERSON')]
[('year ago', 'DATE'), ('joaquin phoenix', 'PERSON'), ('jack nicholson', 'PERSON')]
[]
[('joaquin phoenix joker', 'PERSON'), ('joker jack nicholsons', 'PERSON'), ('phoenix', 'GPE'), ('one', 'CARDINAL')]

**Write your explanations of the constituency parsing tree and dependency parsing tree here (Question 3-2):** 

'''
Write your explanations of the constituency parsing tree and dependency parsing tree here.


**Dependency parsing** - displays grammatical structure where each word represents a node and followed by links to its dependents.
 It displays the relation between words.

**Consituency parsing ** - displays syntactic structure where the words are organized into nested constituents.
Both are helpful in word processing systems and grammar correction.
